In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.chdir('/home/jovyan/work/ufc/')

In [6]:
import pandas as pd
import numpy as np

import os
import time

import click
import joblib
import json

from ruamel.yaml import YAML

import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.calibration import CalibratedClassifierCV, CalibrationDisplay

In [7]:
conf = YAML().load(open('params.yaml'))

In [8]:
DN = conf['train_eval']['report_dn']

df = pd.read_csv(conf['feat_shortlist_sel']['feat_short_fn'])
nfeats = ['event', 'fighter', 'opponent', 'event_day', 'target', 'split']
feats = [it for it in df.columns if not it in nfeats]

In [14]:
fake_split = lambda x: [(x[x.split=='tr'].index.tolist(), x[x.split=='val'].index.tolist())]

# Logreg

In [26]:
from sklearn
model = Pipeline(steps=[('sc', RobustScaler()), ('clf', LogisticRegression(solver='liblinear', random_state=conf['seed']))])


In [27]:
for idx_tr, idx_val in fake_split(df):
    model.fit(df.loc[idx_tr, feats], df.loc[idx_tr, 'target'])

In [28]:
roc_auc_score(df.loc[idx_tr, 'target'], model.predict_proba(df.loc[idx_tr, feats])[:,1]), roc_auc_score(df.loc[idx_val, 'target'], model.predict_proba(df.loc[idx_val, feats])[:,1])

(0.6505486642187126, 0.6930744679611878)